In [1]:
### Installing the required packages if not already installed
packages = ['numpy', 'pandas', 'warnings', 'sqlite3', 'time']

for package in packages:
    try:
        __import__(package)
    except ImportError:
        %pip install {package}


### Start timer
import time
start = time.time()

import numpy as np
import pandas as pd
import warnings
import sqlite3
import os

### Ignoring the warnings
warnings.filterwarnings('ignore')

### Setting working directory
os.chdir('/Users/emilwilliamhansen/Desktop/Master-Thesis/Code')

#### Goal:

Here I will simply make sure all the monthly and daily data alligns with each other. So, we will simply import all data, make sure it has the same stocks and the same periods in it.

In [2]:
### Importing the monthly data
### Connecting to the database
conn = sqlite3.connect('Data/data.db')

### Getting data from the SQL database
monthly_returns = pd.read_sql_query("SELECT * FROM monthly_returns", conn, index_col='index')
daily_returns = pd.read_sql_query("SELECT * FROM daily_returns", conn, index_col='index')


filtered_monthly_returns = pd.read_sql_query("SELECT * FROM filtered_monthly_returns", conn, index_col='index')
filtered_daily_returns = pd.read_sql_query("SELECT * FROM filtered_daily_returns", conn, index_col='index')


factors_monthly = pd.read_sql_query("SELECT * FROM factors_monthly", conn, index_col='date')
factors_daily = pd.read_sql_query("SELECT * FROM factors_daily", conn, index_col='date')
liq_monthly = pd.read_sql_query("SELECT * FROM liq_monthly", conn, index_col='date')
liq_daily = pd.read_sql_query("SELECT * FROM liq_daily", conn, index_col='date')

### Closing the connection
conn.close()

#### Fixing the indexes

In [3]:
monthly_returns.index = pd.to_datetime(monthly_returns.index)
daily_returns.index = pd.to_datetime(daily_returns.index)

filtered_monthly_returns.index = pd.to_datetime(filtered_monthly_returns.index)
filtered_daily_returns.index = pd.to_datetime(filtered_daily_returns.index)

factors_monthly.index = pd.to_datetime(factors_monthly.index)
factors_daily.index = pd.to_datetime(factors_daily.index)
liq_monthly.index = pd.to_datetime(liq_monthly.index)
liq_daily.index = pd.to_datetime(liq_daily.index)

#### Renaming all the dates colums to 'Date'

In [4]:
monthly_returns.index.name = 'Date'
daily_returns.index.name = 'Date'

filtered_monthly_returns.index.name = 'Date'
filtered_daily_returns.index.name = 'Date'

factors_monthly.index.name = 'Date'
factors_daily.index.name = 'Date'
liq_monthly.index.name = 'Date'
liq_daily.index.name = 'Date'

#### Restricting the period to 1980-2023

In [5]:
### Making all returns go from 1980-01-01 to 2023-12-31
monthly_returns = monthly_returns.loc['1980-01-01':'2023-12-31']
daily_returns = daily_returns.loc['1980-01-01':'2023-12-31']

filtered_monthly_returns = filtered_monthly_returns.loc['1980-01-01':'2023-12-31']
filtered_daily_returns = filtered_daily_returns.loc['1980-01-01':'2023-12-31']


factors_monthly = factors_monthly.loc['1980-01-01':'2023-12-31']
factors_daily = factors_daily.loc['1980-01-01':'2023-12-31']
liq_monthly = liq_monthly.loc['1980-01-01':'2023-12-31']
liq_daily = liq_daily.loc['1980-01-01':'2023-12-31']

#### Fixing the shapes

In [6]:
print('Monthly Returns:', monthly_returns.shape)
print('Filtered Monthly Returns:', filtered_monthly_returns.shape)
print('Factors Monthly:', factors_monthly.shape)
print('Liq Monthly:', liq_monthly.shape)

Monthly Returns: (528, 1074)
Filtered Monthly Returns: (528, 995)
Factors Monthly: (528, 5)
Liq Monthly: (491, 4)


In [7]:
liq_monthly = liq_monthly.reindex(monthly_returns.index)
factors_monthly = factors_monthly.reindex(monthly_returns.index)

In [8]:
print('Daily Returns:', daily_returns.shape)
print('Filtered Daily Returns:', filtered_daily_returns.shape)
print('Factors Daily:', factors_daily.shape)
print('Liq Daily:', liq_daily.shape)

Daily Returns: (11041, 1074)
Filtered Daily Returns: (11041, 995)
Factors Daily: (11102, 5)
Liq Daily: (10265, 2)


In [9]:
factors_daily = factors_daily.reindex(daily_returns.index)
liq_daily = liq_daily.reindex(daily_returns.index)

### Reshuffeling the factors

In [10]:
factors_monthly = factors_monthly[['MKT', 'SMB', 'HML', 'UMD', 'rf']]
factors_daily = factors_daily[['MKT', 'SMB', 'HML', 'UMD', 'rf']]

#### Saving all the data again

In [11]:
### Saving all the data to the same database
conn = sqlite3.connect('Data/data.db')

monthly_returns.to_sql('monthly_returns', conn, if_exists='replace')
daily_returns.to_sql('daily_returns', conn, if_exists='replace')

filtered_monthly_returns.to_sql('filtered_monthly_returns', conn, if_exists='replace')
filtered_daily_returns.to_sql('filtered_daily_returns', conn, if_exists='replace')

factors_monthly.to_sql('factors_monthly', conn, if_exists='replace')
factors_daily.to_sql('factors_daily', conn, if_exists='replace')
liq_monthly.to_sql('liq_monthly', conn, if_exists='replace')
liq_daily.to_sql('liq_daily', conn, if_exists='replace')

conn.close()